# Exempel på interaktiv graf med hjälp av bokeh
Nedan är ett exempel på hur man kan göra interaktiva grafer med bokeh i google colab. Det visar sig vara rätt knepigt eftersom `ipywidgets` och en bokeh server inte går att köra från google colab. Det kvarvarande alternativet är då att använda `CustomJS` som är en callback där callbackkoden är skriven i javascript. 

Börja med att importera nödvändiga bibliotek.


In [0]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show
from bokeh.io import output_notebook
import bokeh.palettes

Följande kod är samma som `api.py` som används i väderdata-uppgiften. Först laddas dock datan ner från GitHub, efter det packar `get_station_data()` upp den och sparar den i dicten `data`.

In [0]:
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/smhi/1961.all.ssv.gz --no-verbose
import gzip
import matplotlib.pyplot as plt


def get_station_data():
    result = {}
    with gzip.open('1961.all.ssv.gz', 'rt') as f:
        raw_data = f.read().split('\n\n')
        for s in raw_data:
            station_data = s.split('\n')
            name = ' '.join(station_data[0].split()[1:])
            data = []
            for entry in station_data[1:]:
                y, m, d, t = entry.split()
                data.append((int(y), int(m), int(d), float(t)))
            if(len(name) > 0):
                result[name] = data
    return result


def plot(x=None, y=None, fname="plot.png"):
    if x: 
        plt.plot(x, y)
    else:
        plt.plot(y)
    plt.savefig(fname)
    plt.show()

data = get_station_data()

Nedan är koden som ytterligare förbereder datan och faktiskt plottar.

In [12]:
def data_by_year(year, city_data):
    #Returnar all data för ett år för en stad.
    result = []
    for datum in city_data:
        if datum[0] == year:
            result.append(datum[3])
    return result

def extend_data_dict(city, data_dict):
    #Denna funktionen tar en dict data_dict och lägger till datan för en ny stad till den. Gör så att alla år får exakt 365 mätpunkter genom att lägga till och ta bort värden.
    for i in range(1965, 2017):
        data_dict[city+str(i)]=data_by_year(i, data[city])
        while(len(data_dict[city+str(i)]) < 365):
            data_dict[city+str(i)].append(data_dict[city+str(i)][-1])
        while(len(data_dict[city+str(i)]) > 365):
            data_dict[city+str(i)] = data_dict[city+str(i)][:-1]
    data_dict[city+"days"] = [i+1 for i in range(365)]
    data_dict[city+"T"] = data_dict[city+"1984"]
    return data_dict


def get_layout_plot_cities(cities):
    #returnar en layout som är redo att visas.
    #tar in en lista cities av namnet på städer som ska vara med i plotten.
    
    #lägger till datan i en ColumnDataSource:
    data_dict  = {}
    for city in cities:
        data_dict = extend_data_dict(city, data_dict)
    source = ColumnDataSource(data_dict)
    
    #Skapar själva plotten och lägger till injer i olika färger, och fixar en legend:
    plot = Figure(title="Medeltemperaturer", plot_width=800, plot_height=400, y_range=(-30, 30))
    
    color_scheme = bokeh.palettes.viridis(len(cities))
    for i in range(len(cities)):
        plot.line(cities[i]+"days", cities[i]+"T", source=source, color=color_scheme[i], legend=cities[i])
        
    plot.legend.location="top_left"
    plot.legend.click_policy="hide"
    
    #Skapar CustomJS-objektet som innehåller koden som ska köras när värdet på slidern uppdateras:
    callback=CustomJS(args=dict(source=source), code="""
        var cities = %s;
        var source_data = source.data;
        var f = cb_obj.value;
        for(var i = 0; i < cities.length; ++i){
            var city = cities[i];
            console.log(city)
            var T = source_data[city+'T']
            var days = source_data[city+'days']
            for(var j = 0; j < days.length; ++j){
                T[j] = source_data[city+f.toString()][j]
            }
        }
        source.change.emit();
    """ % cities)
        
    #Skapar en slider och assignar en callback till den:
    slider = Slider(start=1965, end=2016, value=1984, step=1, title="År", callback=callback)
    layout = column(slider, plot)
    return layout
    
#Ange att plotten ska skrivas till notebooken och visa layouten med hjälp av get_layout_plot_cities.
output_notebook()
show(get_layout_plot_cities(["Lund", "Piteå", "Mariestad"]))

Loading BokehJS ...